# CW-Lite/Pro Glitch Exploration

CW-Husky has an internal logic analyzer that is handy for visualizing and validating Husky's glitch generation logic (see `husky_glitch.ipynb`).

CW-lite/pro don't have this internal logic analyzer... but if you have a CW-Husky, you can use *its* logic analyzer to look at the CW-lite/pro-generated glitches! It's a bit hacky but it works.

To do this, use jumper cables to connect these signals between the Husky and CW-lite/pro (on their 20-pin connectors):
- Husky's HS1 to CW-lite/pro's HS2
- Husky's HS2 to CW-lite/pro's HS1
- a Husky ground pin to a CW-lite/pro ground pin

The glitch signals captured in this notebook are digital signals; if you're interested in the actual shape of the glitch output, you won't get that from this; you'll need a good analog oscilloscope instead.

No target needs to be connected for this notebook.

This is also a companion to test_s6_glitch.py, for when visual inspection of glitches is needed.

This notebook assumes familiarity with what's already covered in `husky_glitch.ipynb`; go through that one first.

After you've wired your two ChipWhisperers together, connect them to your computer. If they are the only two ChipWhisperers you have connected, the commands below should establish the connection to each CW without any errors.

In [ ]:
SCOPE="OPENADC"
PLATFORM="CWLITEARM"

In [ ]:
import chipwhisperer as cw
hscope = cw.scope(name='Husky')

# use 'Lite' or 'Pro' as appropriate:
scope = cw.scope(name='Lite')
#scope = cw.scope(name='Pro')

In [ ]:
%run "../Setup_Scripts/Setup_Generic.ipynb"

### Set up Husky:

In our setup, Husky drives the clock which is used by the "target" CW (lite/pro) as its glitch source clock.

The Husky logic analyzer is triggered by *its own* glitch -- not the target CW's glitch -- because calling `hscope.glitch.manual_trigger()` on Husky gives us a handy mechanism to trigger the capture.

The target CW glitch is set to "continuous", so we can see the target glitches even if we're not triggering from them.

In [ ]:
hscope.clock.clkgen_src = 'system'
hscope.clock.clkgen_freq = 10e6
hscope.glitch.enabled = True
hscope.glitch.clk_src = 'pll'
hscope.LA.enabled = True
hscope.LA.clkgen_enabled = True
hscope.LA.oversampling_factor = 50
hscope.LA.capture_group = 'CW 20-pin'
hscope.LA.capture_depth = 512
hscope.LA.trigger_source = "glitch_source"
hscope.io.hs2 = 'clkgen'
hscope.glitch.trigger_src = 'manual'

### Set up target CW (lite/pro):

In [ ]:
scope.glitch.clk_src = 'target'
scope.glitch.repeat = 1
scope.glitch.output = 'glitch_only'
#scope.glitch.trigger_src = 'manual'
scope.glitch.trigger_src = 'continuous'
scope.glitch.offset = 10
scope.glitch.width = 25
scope.glitch.repeat = 10
scope.io.hs2 = 'glitch'

In [ ]:
scope.glitch.resetDCMs()

In [ ]:
assert hscope.LA.locked

### Single capture:

Let's first do a simple single capture.

We can pick arbitrary glitch `offset`, `offset_fine`, `width`, and `width_fine` parameters; start with the values below, then explore other values.

In [ ]:
scope.glitch.offset = 0.8
scope.glitch.width = 15
scope.glitch.width_fine = 0
scope.glitch.offset_fine = 0

In [ ]:
hscope.LA.arm()
hscope.glitch.manual_trigger()
raw = hscope.LA.read_capture_data()
glitchout = hscope.LA.extract(raw, 4)
source    = hscope.LA.extract(raw, 5)

In [ ]:
from bokeh.plotting import figure, show
from bokeh.resources import INLINE
from bokeh.io import output_notebook
from bokeh.models import Span, Legend, LegendItem

import numpy as np
output_notebook(INLINE)

o = figure(plot_width=1800)

xrange = range(len(source))
O1 = o.line(xrange, source + 2, line_color='black')
O2 = o.line(xrange, glitchout + 0, line_color='purple', line_width=2)

legend = Legend(items=[
    LegendItem(label='source clock', renderers=[O1]),
    LegendItem(label='glitch clock output', renderers=[O2]),
])
o.add_layout(legend)

In [ ]:
show(o)

### Interactive glitch visualization: coarse offset and width

Now we step through many coarse width/offset combinations so that we can interactively plot them.

Adjust INCR, START and STOP if desired.

In [ ]:
INCR = 1
START = -48
STOP = 48
STEPS = len(range(START, STOP, INCR)) + 1

allglitchouts = np.zeros((STEPS, STEPS, hscope.LA.capture_depth))
allsources    = np.zeros((STEPS, STEPS, hscope.LA.capture_depth))
actual_offsets = []

from tqdm.notebook import tnrange

scope.glitch.offset = START
scope.glitch.width = START

scope.glitch.offset_fine = 0
scope.glitch.width_fine = 0

for i, o in enumerate(tnrange(START, STOP, INCR)):
    if not o: continue
    scope.glitch.offset = float(o)
    actual_offsets.append(scope.glitch.offset)
    for j, w in enumerate(range(START, STOP, INCR)):
        if not w: continue
        scope.glitch.width = float(w)
        hscope.LA.arm()
        hscope.glitch.manual_trigger()
        raw = hscope.LA.read_capture_data()
        allglitchouts[i][j] = hscope.LA.extract(raw, 4)
        allsources[i][j]    = hscope.LA.extract(raw, 5)


In [ ]:
def update_plot(offset, width):
    S1.data_source.data['y'] = allsources[offset][width] + 2
    S2.data_source.data['y'] = allglitchouts[offset][width] + 0
    push_notebook()

In [ ]:
from ipywidgets import interact, Layout
from bokeh.io import push_notebook
from bokeh.models import Span, Legend, LegendItem

o = 0
w = 0

S = figure(plot_width=1800)

xrange = range(len(allsources[o][w]))
S1 = S.line(xrange, allsources[o][w] + 2, line_color='black')
S2 = S.line(xrange, allglitchouts[o][w] + 0, line_color='purple', line_width=2)


legend = Legend(items=[
    LegendItem(label='Husky source clock', renderers=[S1]),
    LegendItem(label='CW-lite glitch clock output', renderers=[S2]),
])
S.add_layout(legend)

In [ ]:
show(S, notebook_handle=True)

In [ ]:
interact(update_plot, offset=(0, STEPS-1), width=(0, STEPS-1))

### Interactive glitch visualization: fine offset and width

Now we step through many fine width/offset combinations so that we can interactively plot them.

Adjust STEPS if desired, but at 50x oversampling, smaller fine phase adjustments than the defaults here can't be observed.

In [ ]:
# set coarse settings to what you want:
scope.glitch.width = 49.803
scope.glitch.offset = 45

In [ ]:
# Loop over fine offsets:
STEPS = 31
INCR = 511 // STEPS

allglitchouts = np.zeros((STEPS, STEPS, hscope.LA.capture_depth))
allsources    = np.zeros((STEPS, STEPS, hscope.LA.capture_depth))

from tqdm.notebook import tnrange

#scope.glitch.offset = 1.0
#scope.glitch.width = 25.0

scope.glitch.offset_fine = -255
scope.glitch.width_fine = -255

for o in tnrange(STEPS):
    scope.glitch.width_fine = -255
    for w in range(STEPS):
        hscope.LA.arm()
        hscope.glitch.manual_trigger()
        raw = hscope.LA.read_capture_data()
        allglitchouts[o][w] = hscope.LA.extract(raw, 4)
        allsources[o][w]    = hscope.LA.extract(raw, 5)
        scope.glitch.width_fine += INCR
    scope.glitch.offset_fine += INCR

In [ ]:
from ipywidgets import interact, Layout
from bokeh.io import push_notebook
from bokeh.models import Span, Legend, LegendItem

o = 0
w = 0

S = figure(plot_width=1800)

xrange = range(len(allsources[o][w]))
S1 = S.line(xrange, allsources[o][w] + 2, line_color='black')
S2 = S.line(xrange, allglitchouts[o][w] + 0, line_color='purple', line_width=2)


legend = Legend(items=[
    LegendItem(label='Husky source clock', renderers=[S1]),
    LegendItem(label='CW-lite glitch clock output', renderers=[S2]),
])
S.add_layout(legend)

In [ ]:
show(S, notebook_handle=True)

In [ ]:
interact(update_plot, offset=(0, STEPS-1), width=(0, STEPS-1))

# Validation: "Double Glitches" bug

In running the captures above, you will have gotten a bunch of these warnings:

`WARNING:ChipWhisperer Glitch:Negative offsets <-45 may result in double glitches!`

This "double glitches" refers to the situation where `scope.glitch.repeat = ` but yet **two** glitches are produced.

This is a [known issue](https://github.com/newaetech/chipwhisperer/issues/261) which is due to [clock domain crossings](https://en.wikipedia.org/wiki/Clock_domain_crossing). If you're experienced with digital design and multiple clocks, you will appreciate the challenge here: there are several clocks involved in the creation of glitches, and for certain width/offet settings, the relationship of these clocks makes it hard to avoid setup/hold violations. In our case here, the end result is that sometimes, the internal "glitch enable" signal which controls the number of glitches can sometimes last two clock cycles instead of one, which leads to the "double glitch" bug.

ChipWhisperer software warns that double glitches are possible whenever `scope.glitch.offset < -45`, but we can use this setup to find *precisely* which settings result in double glitches. It turns out that, *for a particular FPGA bitfile*, double glitches are highly reproducible. The settings below give double glitches for the FPGA bitfiles used at the time of this writing; if the bitfiles are updated in the future, different results may be obtained. The `test_S6_glitch.py` script can be used to hunt for double glitch parameters.

We need to change a couple of things in our capture setup: we make the target CW output the "glitch enable" signal instead of the glitch itself, and we make the Husky logic analyzer trigger on HS1 (the "glitch enable") instead; `manual_trigger()` is then called on the *target* CW to trigger the capture.

In [ ]:
scope.glitch.output = 'enable_only'
scope.glitch.trigger_src = 'manual'
scope.glitch.repeat = 1
hscope.LA.trigger_source = "HS1"
hscope.LA.oversampling_factor = 40

### Single capture to show a double glitch:

In [ ]:
scope.glitch.offset_fine = 0
scope.glitch.width_fine = 0

scope.glitch.offset = -49.0
#scope.glitch.offset = 10.0
scope.glitch.width = 5

In [ ]:
hscope.LA.arm()
scope.glitch.manual_trigger()
raw = hscope.LA.read_capture_data()
glitchout = hscope.LA.extract(raw, 4)
source    = hscope.LA.extract(raw, 5)
glitchlen = len(np.where(glitchout > 0)[0])
oversamp = hscope.LA.oversampling_factor
cycles = glitchlen/oversamp

assert abs(glitchlen - oversamp) < oversamp / 4, "Double glitch! Glitch enable seen high for %f cycles" % cycles

You should get the assertion error with the default settings. If you change `scope.glitch.offset` to something else (e.g. 10), the assertion should then pass.

In [ ]:
from bokeh.plotting import figure, show
from bokeh.resources import INLINE
from bokeh.io import output_notebook
import numpy as np
output_notebook(INLINE)

o = figure(plot_width=1800)

xrange = range(len(source))
o.line(xrange, source + 2, line_color='black')
o.line(xrange, glitchout + 0, line_color='purple', line_width=2)

In [ ]:
show(o)

Here's an example of looping over several offsets to find which result in double glitches:

In [ ]:
glitches = 1
oversamp = 20
fine_step = 5
desc = ''

scope.glitch.repeat = glitches
failing_offsets = []
maxwidth = 0

#scope.glitch.output = 'glitch_only'
scope.glitch.output = 'enable_only'
scope.glitch.trigger_src = 'manual'
scope.glitch.repeat = 1
hscope.LA.trigger_source = "HS1"
hscope.LA.oversampling_factor = oversamp

maxwidth = 0
good = 0
bad = 0
failing_offsets = []

for i in range(4):
    offset_coarse = -49 + i*0.5
    scope.glitch.offset = offset_coarse
    for offset_fine in range(-255, 255, fine_step):
        scope.glitch.offset_fine = offset_fine
        if not scope.glitch.offset_fine == offset_fine:
            continue
        hscope.LA.arm()
        scope.glitch.manual_trigger()
        raw = hscope.LA.read_capture_data()
        glitchout = hscope.LA.extract(raw, 4)
        glitchlen = len(np.where(glitchout > 0)[0])
        cycles = glitchlen/oversamp

        if glitchlen and (abs(glitchlen/glitches - oversamp) > oversamp/4):
            bad += 1
            failing_offsets.append([scope.glitch.offset, offset_fine])
            if glitchlen > maxwidth:
                maxwidth = glitchlen
        elif glitchlen:
            good += 1

if not failing_offsets:
    print("No double glitches seen!")
else:
    for fail in failing_offsets:
        print("Double glitch with coarse offset: %f, fine offset: %d" % (fail[0], fail[1]))
    print("Maximum glitch length seen (in cycles): %f" % (maxwidth/oversamp))
